# Cropland Data Later - Review

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import pyproj
import rasterio
import rasterstats
import pandas as pd
import json
from rasterio.mask import mask
from rasterio.plot import show
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.colors

## Reading shapefile

In [ ]:
counties = geopandas.read_file("../Data/Images/cb_2017_us_county_500k/cb_2017_us_county_500k.shp")
CA = counties[counties.STATEFP=="06"]

In [ ]:
CA.plot()

## Reading the raster

In [ ]:
r20 = rasterio.open("../Data/Images/CDL_2020_clip_20210316212728_848680512.tif")

In [ ]:
plt.imshow(r20.read(1))

In [ ]:
r20.bounds

The value of bounds attribute is derived from a more fundamental attribute: the dataset’s geospatial transform.

In [ ]:
r20.transform

A dataset’s `transform` is an affine transformation matrix that maps pixel locations in (row, col) coordinates to (x, y) spatial positions

In [ ]:
r20.transform * (0, 0)

Note that these coordinate values are relative to the origin of the dataset’s coordinate reference system (CRS). In this case:

In [ ]:
r20.crs

## Check CRS

In [ ]:
print(r20.crs==CA.crs)
print(r20.crs,CA.crs)

In [ ]:
CA = CA.to_crs(r20.crs)

In [ ]:
CA.plot()

## Clipping

But before, we need to know what is the shape that we will use to perform the clipping!

In [ ]:
CA_State = CA.dissolve(by="STATEFP",aggfunc="sum")

In [ ]:
CA_State.plot()

If we are going to use a geopandas object (our case) to perform the clip, we need to run define this function first. Which will read geopandas geometry column in a way that rasterio can get.

In [ ]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

Now, we are ready to create a NEW RASTER. 

In [ ]:
img_out, img_transform = mask(r20,getFeatures(CA_State))

out_meta = r20.meta.copy()

out_meta['height'] = img_out.shape[1]
out_meta['width'] = img_out.shape[2]
out_meta['transform'] = img_transform

with rasterio.open("./CA20_CDL.tif", "w", **out_meta) as dest:
    dest.write(img_out)

In [ ]:
r20_c = rasterio.open("./CA20_CDL.tif")

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(15, 20));
rasterio.plot.show(r20, ax=ax[0]);
CA_State.plot(ax=ax[1], facecolor='none', edgecolor='black');
rasterio.plot.show(r20_c,ax=ax[2]);

ax[0].set_title("Original Raster (to be clipped)");
ax[1].set_title("Polygon used to clip");
ax[2].set_title("Resulting raster");

ax[0].axis('off');
ax[1].axis('off');
ax[2].axis('off');

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
show(r20_c, ax=ax,cmap="Greens")
CA.plot(ax=ax, facecolor='none', edgecolor='black')

# Reading the variable inside the raster

In [ ]:
r20_c_array = r20_c.read(1)   # The function read from a raster object will give us the data inside it.
r20_c_array 

In [ ]:
pixels = pd.Series(r20_c_array.flatten())   # We can transform that into a Series (DataFrame column)

In [ ]:
pixels

In [ ]:
pixels = pixels[pixels!=0]   # Removing everything that is not zero.

In [ ]:
pixels

In [ ]:
pixels_counts = pixels.value_counts()
pixels_counts.head(10)

In [ ]:
with open("../Data/Images/CDL_dict.json") as f:
    pixel_names = json.load(f)[0]

In [ ]:
pixel_names

## Visualizing 

Let's say that we want to see ONLY grapes.

Remember that up to this point, we have two main objects related to our raster:
- r20_c: The resulting raster after clipping
- r20_c_array: Is the "read" part of the raster (where the data is contained). 


### Step 1: Deciding which type of code we want to see

In [ ]:
# Boolean expression: TRUE if grapes / FALSE if not grape. 
grapes_mask = r20_c_array==69

In [ ]:
# Assigning (replacing) zero to everything that is NOT grape:
r20_c_array[~grapes_mask]=0

### Step 2:  Let's create the raster with the information that we need

In [ ]:
img_out = np.zeros(r20_c_array.shape)  # Dimension of the img_out is the same as the original raster data. 
img_out[grapes_mask] = 1  # Assigning 1 for grapes
img_out = img_out[np.newaxis,:,:].astype(np.uint8)  # Adding a dimension to have a 3-D array

out_meta = r20_c.meta.copy()

out_meta['height'] = img_out.shape[1]
out_meta['width'] = img_out.shape[2]
out_meta['dtype'] = "uint8"
out_meta['transform'] = r20_c.transform

with rasterio.open("./grapes_layer20.tif", "w", **out_meta) as dest:
    dest.write(img_out)

In [ ]:
grapes20 = rasterio.open("./grapes_layer20.tif")

Just to confirm that we are actually getting what we need (just grapes)

In [ ]:
pd.Series(grapes20.read(1).flatten()).value_counts()

In [ ]:
plt.imshow(grapes20.read(1).astype(float), cmap="Blues")

In [ ]:
grapes20.crs == CA.crs

### Step 3: Crate labels for the data inside the raster and plot

In [ ]:
labels = {0:"No-grapes",
          1:"Grapes"}

cmap_grapes = matplotlib.colors.ListedColormap(['white', "C3"])
patches =[mpatches.Patch(color=cmap_grapes(i),label=labels[i]) for i in labels.keys()]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10));
CA.plot(ax=ax, facecolor='none', edgecolor='black');
rasterio.plot.show(grapes20, ax=ax,cmap=cmap_grapes);
plt.legend(handles=patches,fontsize=12);
ax.axis("off");